## Load Data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens-20m-dataset/rating.csv
/kaggle/input/movielens-20m-dataset/link.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/movie.csv


In [2]:
genome_scores_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/genome_scores.csv',
                               usecols=["movieId","tagId","relevance"],
                               dtype={"movieId":"int32","tagId":"int16","relevance":"float32"})

genome_tags_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/genome_tags.csv')
link_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/link.csv')
movie_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')
rating_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv',
                        usecols=["userId","movieId","rating","timestamp"],
                        dtype={"userId":"int32","movieId":"int32","rating":"float32"})
tag_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/tag.csv',
                     usecols=["userId","movieId","tag"],
                     dtype={"userId":"int32","movieId":"int32","tag":"string"})
genome_scores_df

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675
...,...,...,...
11709763,131170,1124,0.58775
11709764,131170,1125,0.01075
11709765,131170,1126,0.01575
11709766,131170,1127,0.11450


**Feature Engineering**

Create genome_mean, genome_max, and genome_std features to summarize genome information for each movie

In [3]:
gfeat = (genome_scores_df.groupby("movieId", sort=False)["relevance"]
         .agg(genome_mean="mean",
              genome_max="max",
              genome_std="std")
         .reset_index())

# strong tag count
strong = (genome_scores_df.assign(strong=(genome_scores_df["relevance"] >= 0.7).astype("int16"))
          .groupby("movieId", sort=False)["strong"].sum()
          .reset_index(name="genome_n_strong_tags"))

gfeat = gfeat.merge(strong, on="movieId", how="left")
gfeat

,movieId,genome_mean,genome_max,genome_std,genome_n_strong_tags
0,1,0.160223,0.99925,0.198524,43
1,2,0.119486,0.98100,0.157490,18
2,3,0.092403,0.97450,0.112546,5
3,4,0.085427,0.97675,0.112448,5
4,5,0.098853,0.96575,0.137117,13
...,...,...,...,...,...
10376,130578,0.061531,0.88325,0.093859,5
10377,130840,0.121182,0.96500,0.151381,13
10378,131013,0.097136,0.98425,0.132048,13
10379,131168,0.136695,0.97300,0.169850,16


In [4]:
genome_scores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int32  
 1   tagId      int16  
 2   relevance  float32
dtypes: float32(1), int16(1), int32(1)
memory usage: 111.7 MB


In [5]:
genome_scores_df.describe()

,movieId,tagId,relevance
count,1.170977e+07,1.170977e+07,1.170977e+07
mean,2.584297e+04,5.645000e+02,1.164834e-01
std,3.467615e+04,3.256254e+02,1.528214e-01
min,1.000000e+00,1.000000e+00,2.500000e-04
25%,2.926000e+03,2.827500e+02,2.425000e-02
50%,6.017000e+03,5.645000e+02,5.650000e-02
75%,4.606200e+04,8.462500e+02,1.415000e-01
max,1.311700e+05,1.128000e+03,1.000000e+00


In [6]:
link_df

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
27273,131254,466713,4436.0
27274,131256,277703,9274.0
27275,131258,3485166,285213.0
27276,131260,249110,32099.0


In [7]:
movie_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [8]:
movie_df.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [9]:
rating_df

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [10]:
rating_df["binary_rating"] = (rating_df["rating"] >= 4.0).astype("int8")
rating_df = rating_df.drop(columns=["rating"])
rating_df

,userId,movieId,timestamp,binary_rating
0,1,2,2005-04-02 23:53:47,0
1,1,29,2005-04-02 23:31:16,0
2,1,32,2005-04-02 23:33:39,0
3,1,47,2005-04-02 23:32:07,0
4,1,50,2005-04-02 23:29:40,0
...,...,...,...,...
20000258,138493,68954,2009-11-13 15:42:00,1
20000259,138493,69526,2009-12-03 18:31:48,1
20000260,138493,69644,2009-12-07 18:10:57,0
20000261,138493,70286,2009-11-13 15:42:24,1


**Feature Engineering**

Create review_year, review_month, and review_dow (Day of the week) as these might have temporal effects on binary_rating

In [11]:
dt = pd.to_datetime(
    rating_df["timestamp"],
    errors="coerce"
)

rating_df["review_year"] = dt.dt.year.astype("int16")
rating_df["review_month"] = dt.dt.month.astype("int8")
rating_df["review_dow"] = dt.dt.dayofweek.astype("int8")

rating_df = rating_df.drop(columns=["timestamp"])

In [12]:
rating_df

,userId,movieId,binary_rating,review_year,review_month,review_dow
0,1,2,0,2005,4,5
1,1,29,0,2005,4,5
2,1,32,0,2005,4,5
3,1,47,0,2005,4,5
4,1,50,0,2005,4,5
...,...,...,...,...,...,...
20000258,138493,68954,1,2009,11,4
20000259,138493,69526,1,2009,12,3
20000260,138493,69644,0,2009,12,0
20000261,138493,70286,1,2009,11,4


In [13]:
rating_df.isnull().sum()

userId           0
movieId          0
binary_rating    0
review_year      0
review_month     0
review_dow       0
dtype: int64

In [14]:
rating_df.duplicated(subset=['userId', 'movieId']).sum()

np.int64(0)

In [15]:
tag_df

,userId,movieId,tag
0,18,4141,Mark Waters
1,65,208,dark hero
2,65,353,dark hero
3,65,521,noir thriller
4,65,592,dark hero
...,...,...,...
465559,138446,55999,dragged
465560,138446,55999,Jason Bateman
465561,138446,55999,quirky
465562,138446,55999,sad


**Feature Engineering**

Create has_user_tag, n_user_tags, and avg_user_tag_len features.
* has_user_tag : If a user includes a tag in their reviews
* n_user_tags : The number of tags a user has included
* avg_user_tag_len: Avergae tag length for each user

In [16]:
tag_df["tag"] = tag_df["tag"].fillna("")

tag_agg = (tag_df
           .assign(tag_len=tag_df["tag"].str.len().astype("int16"))
           .groupby(["userId","movieId"], sort=False)
           .agg(n_user_tags=("tag","size"),
                avg_user_tag_len=("tag_len","mean"))
           .reset_index())

tag_agg["has_user_tag"] = (tag_agg["n_user_tags"] > 0).astype("int8")
tag_agg["n_user_tags"] = tag_agg["n_user_tags"].astype("int16")
tag_agg["avg_user_tag_len"] = tag_agg["avg_user_tag_len"].fillna(0).astype("float32")
tag_agg

,userId,movieId,n_user_tags,avg_user_tag_len,has_user_tag
0,18,4141,1,11.000000,1
1,65,208,1,9.000000,1
2,65,353,1,9.000000,1
3,65,521,1,13.000000,1
4,65,592,1,9.000000,1
...,...,...,...,...,...
174839,138446,3489,3,12.333333,1
174840,138446,7045,2,23.500000,1
174841,138446,7164,2,13.500000,1
174842,138446,55999,4,7.250000,1


In [17]:
df_1 = rating_df.merge(tag_agg, on=['userId','movieId'], how='left') \
                .merge(movie_df, on='movieId', how='left')
df_1

,userId,movieId,binary_rating,review_year,review_month,review_dow,n_user_tags,avg_user_tag_len,has_user_tag,title,genres
0,1,2,0,2005,4,5,NaN,NaN,NaN,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,0,2005,4,5,NaN,NaN,NaN,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,0,2005,4,5,NaN,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,0,2005,4,5,NaN,NaN,NaN,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,0,2005,4,5,NaN,NaN,NaN,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...,...,...,...,...,...
20000258,138493,68954,1,2009,11,4,NaN,NaN,NaN,Up (2009),Adventure|Animation|Children|Drama
20000259,138493,69526,1,2009,12,3,NaN,NaN,NaN,Transformers: Revenge of the Fallen (2009),Action|Adventure|Sci-Fi|IMAX
20000260,138493,69644,0,2009,12,0,NaN,NaN,NaN,Ice Age: Dawn of the Dinosaurs (2009),Action|Adventure|Animation|Children|Comedy|Rom...
20000261,138493,70286,1,2009,11,4,NaN,NaN,NaN,District 9 (2009),Mystery|Sci-Fi|Thriller


**Feature Engineering**

Create movie_year and movie_age features.
* movie_year: extracted from the movie title.
* movie_age: how old the movie is at the review date (in years)

In [18]:
df_1["movie_year"] = (
    df_1["title"]
      .str.extract(r"\((\d{4})\)")
      .fillna(0)
      .astype("int16")
)

df_1['movie_age']=df_1['review_year']-df_1['movie_year']

df_1 = df_1.drop(columns=['title'])
df_1

,userId,movieId,binary_rating,review_year,review_month,review_dow,n_user_tags,avg_user_tag_len,has_user_tag,genres,movie_year,movie_age
0,1,2,0,2005,4,5,NaN,NaN,NaN,Adventure|Children|Fantasy,1995,10
1,1,29,0,2005,4,5,NaN,NaN,NaN,Adventure|Drama|Fantasy|Mystery|Sci-Fi,1995,10
2,1,32,0,2005,4,5,NaN,NaN,NaN,Mystery|Sci-Fi|Thriller,1995,10
3,1,47,0,2005,4,5,NaN,NaN,NaN,Mystery|Thriller,1995,10
4,1,50,0,2005,4,5,NaN,NaN,NaN,Crime|Mystery|Thriller,1995,10
...,...,...,...,...,...,...,...,...,...,...,...,...
20000258,138493,68954,1,2009,11,4,NaN,NaN,NaN,Adventure|Animation|Children|Drama,2009,0
20000259,138493,69526,1,2009,12,3,NaN,NaN,NaN,Action|Adventure|Sci-Fi|IMAX,2009,0
20000260,138493,69644,0,2009,12,0,NaN,NaN,NaN,Action|Adventure|Animation|Children|Comedy|Rom...,2009,0
20000261,138493,70286,1,2009,11,4,NaN,NaN,NaN,Mystery|Sci-Fi|Thriller,2009,0


In [19]:
gfeat

,movieId,genome_mean,genome_max,genome_std,genome_n_strong_tags
0,1,0.160223,0.99925,0.198524,43
1,2,0.119486,0.98100,0.157490,18
2,3,0.092403,0.97450,0.112546,5
3,4,0.085427,0.97675,0.112448,5
4,5,0.098853,0.96575,0.137117,13
...,...,...,...,...,...
10376,130578,0.061531,0.88325,0.093859,5
10377,130840,0.121182,0.96500,0.151381,13
10378,131013,0.097136,0.98425,0.132048,13
10379,131168,0.136695,0.97300,0.169850,16


In [20]:
df_1 = df_1.merge(gfeat, on=['movieId'], how='left')
df_1

,userId,movieId,binary_rating,review_year,review_month,review_dow,n_user_tags,avg_user_tag_len,has_user_tag,genres,movie_year,movie_age,genome_mean,genome_max,genome_std,genome_n_strong_tags
0,1,2,0,2005,4,5,NaN,NaN,NaN,Adventure|Children|Fantasy,1995,10,0.119486,0.98100,0.157490,18.0
1,1,29,0,2005,4,5,NaN,NaN,NaN,Adventure|Drama|Fantasy|Mystery|Sci-Fi,1995,10,0.181251,0.99800,0.216655,55.0
2,1,32,0,2005,4,5,NaN,NaN,NaN,Mystery|Sci-Fi|Thriller,1995,10,0.198128,0.99825,0.233735,68.0
3,1,47,0,2005,4,5,NaN,NaN,NaN,Mystery|Thriller,1995,10,0.200519,0.99675,0.240812,80.0
4,1,50,0,2005,4,5,NaN,NaN,NaN,Crime|Mystery|Thriller,1995,10,0.231736,0.99875,0.249365,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20000258,138493,68954,1,2009,11,4,NaN,NaN,NaN,Adventure|Animation|Children|Drama,2009,0,0.178547,0.99475,0.217788,59.0
20000259,138493,69526,1,2009,12,3,NaN,NaN,NaN,Action|Adventure|Sci-Fi|IMAX,2009,0,0.128681,0.99850,0.189206,39.0
20000260,138493,69644,0,2009,12,0,NaN,NaN,NaN,Action|Adventure|Animation|Children|Comedy|Rom...,2009,0,0.089562,0.98500,0.138437,17.0
20000261,138493,70286,1,2009,11,4,NaN,NaN,NaN,Mystery|Sci-Fi|Thriller,2009,0,0.195345,0.99900,0.213518,47.0


**Feature Engineering**

Create dummy variables for all movie genres

In [21]:
genres_dummies = (
    df_1['genres']
    .str.get_dummies(sep='|')
)
genres_dummies

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20000258,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
20000259,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
20000260,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
20000261,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0


Export this final dataframe to be used in modeling in parquet to consume less storage in the modeling notebook.

In [22]:
df = pd.concat(
    [df_1.drop(columns='genres'), genres_dummies.drop(columns='(no genres listed)')],
    axis=1
)
df.to_parquet(
    "final_features.parquet",
    index=False
)